# Exploring Ebay Car Sales Data

The aim of the project is to clean and analyze the dataset of used cars on Ebay Kleinanzeigen, a classifieds section of the German eBay website. The dataset was originally downloaded from Kaggle and modified by DataQuest. According to DataQuest:
1. They sampled 50,000 data points from the full dataset.
2. They dirtied the dataset a bit to more closely resemble what you would expect from a scraped dataset (the version uploaded to Kaggle was cleaned to be easier to work with)

### Data Export and Overview

Let's start with importing required libraries and and exporing the dataset.

In [1]:
import pandas as pd
import numpy as np

# The CSV file is encoded in Latin-1 instead of UTF-1, so we have to specify the encoding parameter
dataset = pd.read_csv('data/autos.csv', encoding = 'Latin-1')

In [2]:
dataset.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

We can see that our dataset:
- Has 50000 rows and 20 columns. 
- There are columns of two types - integer and object(most likely string). 
- There are some columns where values are missing - vehicleType, gearbox, model, fuelTypeand notRepairedDamage. 
- Our dataset uses 7.6+ MB of memory.
- Column labels use camelcase instead of Python's preferred snakecase

### Data Cleanse
Let's convert the column names from camelcase to snakecase and reword some of the column names based on the data dictionary to be more descriptive.

In [4]:
columns_dict = {
    'dateCrawled': 'date_crawled',
    'offerType': 'offer_type',
    'vehicleType': 'vehicle_type',
    'yearOfRegistration': 'registration_year',
    'powerPS': 'power_ps',
    'monthOfRegistration': 'registration_month',
    'notRepairedDamage': 'unrepaired_damage',
    'dateCreated': 'ad_created',
    'nrOfPictures': 'nr_of_pictures',
    'postalCode': 'postal_code',
    'lastSeen': 'last_seen'
}

dataset = dataset.rename(columns = columns_dict)
dataset.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuelType,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


As we can see, the titles of columns were successfully changed.

Now let's do some basic data exploration to determine what other cleaning tasks need to be done. Initially we will look for: - Text columns where all or almost all values are the same. These can often be dropped as they don't have useful information for analysis.

Firstly, let's look at descriptive statistics for all columns.

In [5]:
dataset.describe(include = np.object)

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,gearbox,model,odometer,fuelType,brand,unrepaired_damage,ad_created,last_seen
count,50000,50000,50000,50000,50000,50000,44905,47320,47242,50000,45518,50000,40171,50000,50000
unique,48213,38754,2,2,2357,2,8,2,245,13,7,40,2,76,39481
top,2016-03-21 20:37:19,Ford_Fiesta,privat,Angebot,$0,test,limousine,manuell,golf,"150,000km",benzin,volkswagen,nein,2016-04-03 00:00:00,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,36993,4024,32424,30107,10687,35232,1946,8


In [6]:
dataset.describe(include = np.number)

,registration_year,power_ps,registration_month,nr_of_pictures,postal_code
count,50000.000000,50000.000000,50000.000000,50000.0,50000.000000
mean,2005.073280,116.355920,5.723360,0.0,50813.627300
std,105.712813,209.216627,3.711984,0.0,25779.747957
min,1000.000000,0.000000,0.000000,0.0,1067.000000
25%,1999.000000,70.000000,3.000000,0.0,30451.000000
50%,2003.000000,105.000000,6.000000,0.0,49577.000000
75%,2008.000000,150.000000,9.000000,0.0,71540.000000
max,9999.000000,17700.000000,12.000000,0.0,99998.000000


Let's take a look at seller and offer_type columns. Their top values are privat and Angebot, and the frequency of those is 4999. For our analysis, we are not interested in the columns where almost all values are the same. The other column that can be dropped without affecting our analysis is nr_of_pictures - all values are 0.

Let's drop those.

In [7]:
dataset = dataset.drop(['seller', 'offer_type', 'nr_of_pictures'], axis = 1)
dataset.describe(include = np.object)

,date_crawled,name,price,abtest,vehicle_type,gearbox,model,odometer,fuelType,brand,unrepaired_damage,ad_created,last_seen
count,50000,50000,50000,50000,44905,47320,47242,50000,45518,50000,40171,50000,50000
unique,48213,38754,2357,2,8,2,245,13,7,40,2,76,39481
top,2016-03-21 20:37:19,Ford_Fiesta,$0,test,limousine,manuell,golf,"150,000km",benzin,volkswagen,nein,2016-04-03 00:00:00,2016-04-07 06:17:27
freq,3,78,1421,25756,12859,36993,4024,32424,30107,10687,35232,1946,8


We can see that dataset contains some numeric data stored as text - price and odometer. Let's change the type of the columns to the numeric and change the titles of the columns to make it more clear in terms of understanding what are columns about.

In [8]:
# Clean and transform price
price = dataset['price']
price = price.str.replace('$', '')
price = price.str.replace(',', '')
dataset['price'] = price.astype('int64')
dataset = dataset.rename(columns = {'price': 'price_dollars'})

# Clean and transform odometer
odometer = dataset['odometer']
odometer = odometer.str.replace('km', '')
odometer = odometer.str.replace(',', '')
dataset['odometer'] = odometer.astype('int64')
dataset = dataset.rename(columns = {'odometer': 'odometer_km'})

In [9]:
dataset.describe(include = np.object)

,date_crawled,name,abtest,vehicle_type,gearbox,model,fuelType,brand,unrepaired_damage,ad_created,last_seen
count,50000,50000,50000,44905,47320,47242,45518,50000,40171,50000,50000
unique,48213,38754,2,8,2,245,7,40,2,76,39481
top,2016-03-21 20:37:19,Ford_Fiesta,test,limousine,manuell,golf,benzin,volkswagen,nein,2016-04-03 00:00:00,2016-04-07 06:17:27
freq,3,78,25756,12859,36993,4024,30107,10687,35232,1946,8


In [10]:
dataset.describe(include = np.number)

,price_dollars,registration_year,power_ps,odometer_km,registration_month,postal_code
count,5.000000e+04,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,9.840044e+03,2005.073280,116.355920,125732.700000,5.723360,50813.627300
std,4.811044e+05,105.712813,209.216627,40042.211706,3.711984,25779.747957
min,0.000000e+00,1000.000000,0.000000,5000.000000,0.000000,1067.000000
25%,1.100000e+03,1999.000000,70.000000,125000.000000,3.000000,30451.000000
50%,2.950000e+03,2003.000000,105.000000,150000.000000,6.000000,49577.000000
75%,7.200000e+03,2008.000000,150.000000,150000.000000,9.000000,71540.000000
max,1.000000e+08,9999.000000,17700.000000,150000.000000,12.000000,99998.000000


There are three more columns that require more investigation - price_dollars (some prices are 0), power_ps (max 17,700 while in real world is around 2,000), registation_year (we have year min year 1000 and max - 9999) and registration_month (min month is 0)

Let's start with investigating price_dollars column. 

In [11]:
dataset['price_dollars'].describe()

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price_dollars, dtype: float64

We can see price column contains 50000 records and no null values. The minimum value is 0 and maximum is 99999999. We may assume that there will be no exclusive (most expensive) cars sold on ebay market, so there are no cars sold for more than 500000. Also we can assume that there are no cars sold for less than 300

In [12]:
dataset['price_dollars'].value_counts().sort_index(ascending = 'False')

0           1421
1            156
2              3
3              1
5              2
8              1
9              1
10             7
11             2
12             3
13             2
14             1
15             2
17             3
18             1
20             4
25             5
29             1
30             7
35             1
40             6
45             4
47             1
49             4
50            49
55             2
59             1
60             9
65             5
66             1
            ... 
151990         1
155000         1
163500         1
163991         1
169000         1
169999         1
175000         1
180000         1
190000         1
194000         1
197000         1
198000         1
220000         1
250000         1
259000         1
265000         1
295000         1
299000         1
345000         1
350000         1
999990         1
999999         2
1234566        1
1300000        1
3890000        1
10000000       1
11111111       2
12345678      

In [13]:
dataset = dataset[dataset["price_dollars"].between(1000,500000)]

In [14]:
dataset['price_dollars'].describe()

count     38626.000000
mean       7255.376275
std        9698.439853
min        1000.000000
25%        2200.000000
50%        4350.000000
75%        8950.000000
max      350000.000000
Name: price_dollars, dtype: float64

Now the descriptive stats of prices seems to be more or less fine:
- We have no prices smaller than 1000
- We have no prices higher than 500000
- The lowest price is 1000
- The highest price is 350000
- The average price is 7255

Now, let's do the same procedure with power_ps column as it includes 0 values and values that are too high to be real.

In [15]:
dataset['power_ps'].describe()

count    38626.000000
mean       128.915653
std        215.891372
min          0.000000
25%         80.000000
50%        116.000000
75%        160.000000
max      17700.000000
Name: power_ps, dtype: float64

In [16]:
dataset['power_ps'].value_counts().sort_index(ascending = 'False')

0        2938
1           1
2           2
3           2
4           3
5          11
6           1
8           1
9           1
10          2
11          4
14          1
15          5
18          6
19          2
20          3
23          2
24          1
25          2
26         16
27          5
29          3
30          2
33          3
34         26
37          1
38          2
39          8
40         14
41         56
         ... 
1300        1
1367        1
1398        1
1400        2
1401        1
1405        1
1704        1
1771        1
1779        1
1781        1
1793        1
1796        1
1800        1
1986        1
1998        2
2018        1
2729        1
3500        1
4400        1
5867        1
6045        1
6226        1
6512        1
7511        1
9011        1
14009       1
15001       1
16011       1
16312       1
17700       1
Name: power_ps, Length: 426, dtype: int64

According to the https://www.hotcars.com/highest-and-lowest-horsepower-domestic-cars/, the most powerful car has 808 - 850 PS ho and the least - 98 HP. Let's stick to these figure and remove all rows that are not in this range. 

In [17]:
dataset = dataset[dataset["power_ps"].between(99,850)]

In [18]:
dataset['power_ps'].describe()

count    25989.000000
mean       157.816499
std         59.169482
min         99.000000
25%        116.000000
50%        141.000000
75%        177.000000
max        850.000000
Name: power_ps, dtype: float64

In [19]:
dataset['power_ps'].value_counts().sort_index(ascending = 'False')

99       75
100     112
101    1337
102     702
103      98
104       9
105    1327
106      35
107     105
108      22
109     590
110     600
111      81
112      15
113      93
114     100
115     277
116    1373
117      26
118     128
119       7
120     456
121      34
122     630
123       7
124      30
125     782
126      72
127       4
128      61
       ... 
507       5
510       3
514       3
515       1
517       1
521       2
525       4
530       1
544       1
551       2
555       2
559       2
560       4
564       1
571       1
579       2
585       1
587       1
600       3
610       2
612       1
630       1
640       1
650       1
678       1
682       1
696       1
740       1
800       1
850       1
Name: power_ps, Length: 298, dtype: int64

Now the descriptive stats of power_ps seems to be more or less fine:
- We have no power smaller than 99 PS
- We have no power higher than 850 PS
- The average power is 157 PS

The last columns from where we need to remove outliers are registration_year and registration_month. The rule is that year must be not higher than 2019 and not lower than X (will see what we have in data set), and the month should be in between 1 and 12.

Let's start with the registration year.

In [20]:
dataset['registration_year'].value_counts().sort_index(ascending = 'False')

1950       1
1954       1
1955       1
1957       1
1959       3
1960       1
1961       2
1963       2
1964       2
1965       7
1966      10
1967      12
1968      15
1969       6
1970       6
1971       6
1972      11
1973       8
1974       8
1975       7
1976       8
1977      13
1978      18
1979      18
1980      19
1981      16
1982      19
1983      29
1984      18
1985      40
        ... 
1992     143
1993     155
1994     169
1995     291
1996     344
1997     572
1998     854
1999    1105
2000    1184
2001    1314
2002    1383
2003    1608
2004    1796
2005    1918
2006    1909
2007    1661
2008    1633
2009    1395
2010    1156
2011    1292
2012     964
2013     614
2014     501
2015     298
2016     318
2017     514
2018     182
2019       1
5000       1
9000       1
Name: registration_year, Length: 66, dtype: int64

We can see that there are only two outliers left - 5000 and 9000. Let's remove those

In [21]:
dataset = dataset[dataset["registration_year"] < 5000]

In [22]:
dataset['registration_year'].describe()

count    25987.000000
mean      2004.859661
std          6.488803
min       1950.000000
25%       2001.000000
50%       2005.000000
75%       2009.000000
max       2019.000000
Name: registration_year, dtype: float64

All seems good. The smallest year is 1950 and the largest year is 2019. Let's move on to regiration month.

In [23]:
dataset['registration_month'].value_counts().sort_index(ascending = 'False')

0     1126
1     1803
2     1616
3     2715
4     2241
5     2256
6     2423
7     2233
8     1721
9     1945
10    2094
11    1960
12    1854
Name: registration_month, dtype: int64

We have 1126 values of 0 while months should in the range of 1 - 12.

In [24]:
dataset = dataset[dataset["registration_month"].between(1, 12)]

In [25]:
dataset['registration_month'].describe()

count    24861.000000
mean         6.420257
std          3.344435
min          1.000000
25%          4.000000
50%          6.000000
75%          9.000000
max         12.000000
Name: registration_month, dtype: float64

Now, let's calculate distributrion for registration year to see what years of registration are most and leat popular.

In [26]:
dataset['registration_year'].value_counts(normalize=True, dropna=False)

2006    0.074615
2005    0.074132
2004    0.070110
2007    0.065001
2008    0.064358
2003    0.061784
2009    0.055187
2002    0.053055
2011    0.051607
2001    0.049274
2010    0.046137
2000    0.044447
1999    0.040988
2012    0.038333
1998    0.031656
2013    0.024617
1997    0.021278
2014    0.020072
2017    0.017377
1996    0.012872
2015    0.011866
1995    0.010699
2016    0.010418
1994    0.006355
2018    0.006315
1993    0.005470
1992    0.005350
1991    0.004787
1990    0.003580
1989    0.002816
          ...   
1986    0.000845
1982    0.000724
1979    0.000724
1987    0.000684
1980    0.000644
1978    0.000644
1981    0.000644
1984    0.000603
1968    0.000523
1967    0.000483
1977    0.000442
1972    0.000442
1966    0.000402
1974    0.000322
1976    0.000282
1965    0.000282
1973    0.000282
1975    0.000282
1971    0.000241
1970    0.000241
1969    0.000201
1959    0.000121
1961    0.000080
1963    0.000080
1964    0.000080
1955    0.000040
1954    0.000040
1950    0.0000

As we can see from the distribution, the most popular years are 2006, 2005 and 2004 and least popular are 1960, 2019, 1950. 1954 and 1955. 

Now, let's check the period of time when adverts were created.

In [27]:
ad_created = dataset['ad_created'].str[:10]
print(ad_created.value_counts(normalize=True, dropna=False).sort_index())

2015-08-10    0.000040
2016-01-03    0.000040
2016-01-07    0.000040
2016-01-10    0.000080
2016-01-13    0.000040
2016-01-14    0.000040
2016-01-16    0.000040
2016-01-22    0.000040
2016-01-27    0.000040
2016-01-29    0.000040
2016-02-01    0.000040
2016-02-02    0.000080
2016-02-05    0.000080
2016-02-07    0.000040
2016-02-12    0.000080
2016-02-14    0.000040
2016-02-17    0.000040
2016-02-18    0.000040
2016-02-19    0.000080
2016-02-21    0.000040
2016-02-22    0.000040
2016-02-23    0.000121
2016-02-24    0.000080
2016-02-25    0.000121
2016-02-26    0.000040
2016-02-27    0.000080
2016-02-28    0.000362
2016-02-29    0.000161
2016-03-01    0.000080
2016-03-02    0.000121
                ...   
2016-03-09    0.031938
2016-03-10    0.032621
2016-03-11    0.032903
2016-03-12    0.037006
2016-03-13    0.017819
2016-03-14    0.035719
2016-03-15    0.033104
2016-03-16    0.029886
2016-03-17    0.029524
2016-03-18    0.013515
2016-03-19    0.032420
2016-03-20    0.039419
2016-03-21 

Adverts were created in the period of time between 2015-08-10 and 2016-04-07. Based on that, we can assume that we made a mistake when we were cleaning registration year. We removed rows that were higher than 2019 (current year), but adverts were added in 2016. Let's fix that problem by removing years that are higher than 2016.

In [28]:
dataset = dataset[dataset['registration_year'] <= 2016]
dataset['registration_year'].describe()

count    24271.000000
mean      2004.630876
std          6.187178
min       1950.000000
25%       2001.000000
50%       2005.000000
75%       2009.000000
max       2016.000000
Name: registration_year, dtype: float64

Finally our data is cleaned and all outliers are removed. We can move to the analysis part.

### Data analysis
In this section we are going to calculate average prices for:
- Each car brand
- Each odometer value
- Each registration year

Also we are going to calculate average mileage for:
- Each car brand
- Each registration year

Firstly, lets' write a function to calculate the average price for the list of unique items, it could be brands, odometer values or anything else. And another function to print dictionaries. 

In [29]:
def get_average (avg, by):
    avg_dict = {}
    unique_items = dataset[by].unique()
    
    for item in unique_items:
        selected_rows = dataset[dataset[by] == item]
        total_number = selected_rows.shape[0]
        total = selected_rows[avg].sum()
        mean = round(total/total_number)
        avg_dict[item] = mean
    return avg_dict

def print_avg (avg_dict, avg_title, by_title):
    print ("{:<15} {:<15}".format(by_title, avg_title))
    
    for key, value in sorted(avg_dict.items(), key=lambda item: item[1]):
        print ("{:<15} {:<15}".format(key, avg_dict[key]))

### Average price by brand

In [30]:
avg_price_by_brand = get_average('price_dollars', 'brand')
print_avg(avg_price_by_brand, 'Price', 'Brand')

Brand           Price          
daewoo          1578.0         
rover           3143.0         
chrysler        4629.0         
saab            4717.0         
daihatsu        4742.0         
fiat            4833.0         
renault         4863.0         
peugeot         4873.0         
opel            5218.0         
alfa_romeo      5237.0         
mitsubishi      5816.0         
citroen         6084.0         
mazda           6142.0         
volvo           6322.0         
honda           6400.0         
ford            6528.0         
smart           6648.0         
subaru          7245.0         
seat            7274.0         
toyota          7340.0         
lancia          7420.0         
suzuki          7478.0         
hyundai         7504.0         
kia             8211.0         
nissan          8635.0         
volkswagen      8656.0         
skoda           8685.0         
dacia           8825.0         
chevrolet       9468.0         
bmw             9491.0         
mercedes

We can see that th cheapest brand is Daewoo and the most expensive one is Porsche. 

### Average price by odometer value

In [31]:
avg_price_by_odometer = get_average('price_dollars', 'odometer_km')
print_avg(avg_price_by_odometer, 'Price', 'Odometer (km)')

Odometer (km)   Price          
150000          5814.0         
125000          9512.0         
100000          12313.0        
90000           12550.0        
80000           13954.0        
70000           15408.0        
60000           17233.0        
50000           18955.0        
40000           20551.0        
5000            21933.0        
30000           22788.0        
20000           25451.0        
10000           29609.0        


The result is the same as we expected - cars with huge mileage cost less and cars with small mileage cost expensive. The only wierd value we face is for 5000km, for some reason the price for cars with 5000km mileage is higher than for cars with 30000km, 20000km and 10000km mileage.

### Average price by registration year

In [32]:
avg_price_by_registration_year = get_average('price_dollars', 'registration_year')
print_avg(avg_price_by_registration_year, 'Price', 'Registration Year')

Registration Year Price          
1999            2912.0         
1997            3266.0         
2000            3360.0         
1996            3514.0         
1998            3569.0         
2001            3941.0         
1995            4033.0         
2002            4153.0         
2003            4683.0         
1993            4888.0         
1994            5014.0         
1989            5300.0         
2004            5612.0         
1991            5810.0         
1992            6357.0         
2005            6441.0         
1988            6700.0         
1990            7353.0         
1984            7360.0         
2006            7719.0         
1986            8479.0         
1980            8647.0         
1973            9299.0         
1985            9349.0         
2007            9474.0         
1950            10000.0        
1975            10343.0        
1982            10425.0        
1987            10425.0        
1981            11050.0        
2016  

From this table we can see that the cheapest cars were registered in around 2000s and the most expensive cars are old cars or new cars - 1955, 1977 or 2015.

### Average mileage by brand

In [33]:
avg_mileage_by_brand = get_average('odometer_km', 'brand')
print_avg(avg_mileage_by_brand, 'Mileage', 'Brand')

Brand           Mileage        
dacia           79722.0        
mini            84624.0        
porsche         98127.0        
sonstige_autos  104692.0       
hyundai         104978.0       
chevrolet       105000.0       
smart           105556.0       
suzuki          110000.0       
kia             110936.0       
nissan          111579.0       
skoda           112943.0       
toyota          114222.0       
seat            117874.0       
fiat            118046.0       
land_rover      118614.0       
mazda           119021.0       
honda           119424.0       
citroen         119808.0       
daihatsu        120000.0       
mitsubishi      121349.0       
peugeot         121560.0       
lancia          121667.0       
ford            123703.0       
renault         124415.0       
volkswagen      124511.0       
opel            125532.0       
jaguar          125574.0       
subaru          126111.0       
audi            126797.0       
jeep            127606.0       
mercedes

As we can see, Dacia and Mini cars have the smallest mileage and Daewoo and Saab have the largest one.

### Average mileage by regitration year

In [34]:
avg_mileage_by_registration_year = get_average('odometer_km', 'registration_year')
print_avg(avg_mileage_by_registration_year, 'Mileage', 'Registration Year')

Registration Year Mileage        
1950            5000.0         
2015            17576.0        
2014            34228.0        
1971            46667.0        
1975            54286.0        
2013            57361.0        
1965            68571.0        
1964            70000.0        
2012            77513.0        
1969            80000.0        
1960            80000.0        
1970            85000.0        
1967            85417.0        
1968            85769.0        
1974            89375.0        
1955            90000.0        
2011            93359.0        
1972            96364.0        
1980            102188.0       
1966            104000.0       
1976            104286.0       
1984            105667.0       
2010            106356.0       
1978            110000.0       
1977            112273.0       
1963            115000.0       
2009            116272.0       
1981            118125.0       
2016            120907.0       
1979            121111.0       
1959  

The results are mostly same as we expected. Cars that are registered in around 2000 have the highest mileage and old/new cars have smaller mileage. 

### Combining average price and average mileage by brand

But what if we would like to check if there is a visible link between average price by brand and average mileage by brand. We will combine these two columns in one dataframe and display that.

In [35]:
apbb_series = pd.Series(avg_price_by_brand)
ambb_series = pd.Series(avg_mileage_by_brand)

In [36]:
df = pd.DataFrame(apbb_series, columns=['mean_price'])
df['mean_mileage'] = ambb_series
df.sort_values(by = ['mean_price'], inplace = True)
df

,mean_price,mean_mileage
daewoo,1578.0,137857.0
rover,3143.0,131136.0
chrysler,4629.0,131308.0
saab,4717.0,141042.0
daihatsu,4742.0,120000.0
fiat,4833.0,118046.0
renault,4863.0,124415.0
peugeot,4873.0,121560.0
opel,5218.0,125532.0
alfa_romeo,5237.0,129300.0


In this case it is quite difficult to say how the price depends on the mileage, but we can assume that in the case if we will compare those against one brand and one model, as lower mileage car will have, as higher price will be. Let's confirm that on BMW brand - BMW 5er. 

In [37]:
new_dataset = dataset[['brand', 'model', 'odometer_km', 'registration_year', 'price_dollars']]

new_dataset = new_dataset[(new_dataset['brand'] == 'bmw') & (new_dataset['model'] == '5er')]
new_dataset.sort_values(by = ['price_dollars'], inplace = True)
new_dataset

,brand,model,odometer_km,registration_year,price_dollars
23916,bmw,5er,150000,1999,1000
34807,bmw,5er,150000,1997,1000
11374,bmw,5er,150000,1998,1000
36712,bmw,5er,150000,2016,1000
23408,bmw,5er,150000,1997,1000
30618,bmw,5er,150000,1997,1000
5595,bmw,5er,150000,1997,1000
31287,bmw,5er,150000,1994,1000
1771,bmw,5er,150000,1997,1000
15495,bmw,5er,150000,1999,1050


Based on the printed dataframe, we can confirm that there is a strong relationship between registration year, mileage and price. New cars with small mileage are most expensive and old cars with large mileage are cheapest.

## Conclusion

However, a car price depends on much more factors than only these two, there is a strong correlation between price, year of registration and mileage.